In [ ]:

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


x_train, x_test = x_train / 255.0, x_test / 255.0


x_train = x_train.reshape(-1, 28*28)
x_test = x_test.reshape(-1, 28*28)


baseline_model = models.Sequential([
    layers.Dense(256, activation='relu', input_shape=(784,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

baseline_model.compile(optimizer='adam',
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

baseline_history = baseline_model.fit(x_train, y_train,
                                      validation_data=(x_test, y_test),
                                      epochs=10, batch_size=128, verbose=0)

l2_model = models.Sequential([
    layers.Dense(256, activation='relu', input_shape=(784,),
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(10, activation='softmax')
])

l2_model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

l2_history = l2_model.fit(x_train, y_train,
                          validation_data=(x_test, y_test),
                          epochs=10, batch_size=128, verbose=0)


dropout_model = models.Sequential([
    layers.Dense(256, activation='relu', input_shape=(784,)),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])

dropout_model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

dropout_history = dropout_model.fit(x_train, y_train,
                                    validation_data=(x_test, y_test),
                                    epochs=10, batch_size=128, verbose=0)


def plot_metrics(histories, metric='accuracy'):
    plt.figure(figsize=(12, 5))
    for name, history in histories.items():
        plt.plot(history.history[metric], label=f'{name} Train')
        plt.plot(history.history[f'val_{metric}'], '--', label=f'{name} Val')
    plt.title(f'Model {metric.capitalize()} Comparison')
    plt.xlabel('Epochs')
    plt.ylabel(metric.capitalize())
    plt.legend()
    plt.grid(True)
    plt.show()

plot_metrics({
    'Baseline': baseline_history,
    'L2 Regularization': l2_history,
    'Dropout': dropout_history
}, 'accuracy')


plot_metrics({
    'Baseline': baseline_history,
    'L2 Regularization': l2_history,
    'Dropout': dropout_history
}, 'loss')


print("Final Test Accuracy:")
print(f"Baseline: {baseline_model.evaluate(x_test, y_test, verbose=0)[1]:.4f}")
print(f"L2 Regularization: {l2_model.evaluate(x_test, y_test, verbose=0)[1]:.4f}")
print(f"Dropout: {dropout_model.evaluate(x_test, y_test, verbose=0)[1]:.4f}")